<a href="https://colab.research.google.com/github/WrongMedal/ML_proj_Orthogonal_Re-Basin/blob/main/MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#MLP

###Install

In [ ]:
!pip install pytorch-lightning --quiet
!pip install wandb -qU


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 828.2/828.2 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.0/983.0 kB 31.3 MB/s eta 0:00:00


###Import

In [ ]:
#General
import pytorch_lightning as pl
import torch
import torch.nn as nn
import torch.nn.functional as F

#Data
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torchvision.datasets import MNIST
from torchvision import transforms

#Valutazioni e grafica
from torchmetrics import Accuracy
import seaborn as sns
import matplotlib as plt

#Logging & Callbacks
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks import ModelCheckpoint
import wandb
from pytorch_lightning.loggers import WandbLogger
wandb.login()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: wrongmedal (wrongmedal_wb) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

###MLP

In [ ]:
class MLP(pl.LightningModule):
    def __init__(self):
        super().__init__()

        self.train_acc = Accuracy(task="multiclass", num_classes=10)
        self.valid_acc = Accuracy(task="multiclass", num_classes=10)
        self.test_acc = Accuracy(task="multiclass", num_classes=10)

        input_size = 28 * 28        #Dim of pictures in MNIST including channels
        output_size = 10            #Num of classes
        layers = [nn.Flatten()]
        hidd_layers_dim = (128, 64, 64)

        for hidd_dim in hidd_layers_dim:
            layers.append(nn.Linear(input_size, hidd_dim))
            layers.append(nn.ReLU())
            layers.append(nn.BatchNorm1d(hidd_dim))
            layers.append(nn.Dropout(0.25))
            input_size = hidd_dim

        layers.append(nn.Linear(input_size, output_size))
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

    #Seguendo paradigma lightning
    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)   #softmax è implicito
        preds = torch.argmax(logits, dim=1) #la classe più prob.

        self.train_acc.update(preds, y)
        self.log("train_loss", loss, prog_bar=True)
        return loss #backprop, agg.gradiente e utilizzo optimizer impliciti con lightning

    def on_training_epoch_end(self):
        self.log("train_acc", self.train_acc.compute(), prog_bar=True)
        self.train_acc.reset()

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        preds = torch.argmax(logits, dim=1)

        self.valid_acc.update(preds, y)
        self.log("valid_loss", loss, prog_bar=True)
        return loss

    def on_validation_epoch_end(self):
        self.log("valid_acc", self.valid_acc.compute(), prog_bar=True)
        self.valid_acc.reset()

    def test_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        preds = torch.argmax(logits, dim=1)
        self.test_acc.update(preds, y)
        self.log("test_loss", loss, prog_bar=True)
        return loss

    def on_test_epoch_end(self):
        self.log("test_acc", self.test_acc.compute(), prog_bar=True)
        self.test_acc.reset()

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.001)

###MNISTDataModule

In [ ]:
class MNISTDataModule(pl.LightningDataModule):
    def __init__(self, seme, data_path='./'):
        super().__init__()
        self.data_path = data_path #Percorso in cui scarico dataset
        self.seme = seme
        self.transform = transforms.Compose([transforms.ToTensor()])

    def prepare_data(self):
        MNIST(root=self.data_path, download=True)

    def setup(self, stage=None):
        MNIST_all = MNIST(
            root=self.data_path,
            train=True,
            transform=self.transform,
            download=False
        )

        generator = torch.Generator().manual_seed(self.seme)
        full_split = random_split(MNIST_all, [55000, 5000], generator=generator) # Corrected split lengths

        self.train = full_split[0]
        self.val = full_split[1]

        self.test = MNIST(
            root=self.data_path,
            train=False,
            transform=self.transform,
            download=False
        )

    def train_dataloader(self):
        return DataLoader(self.train, batch_size=64, num_workers=2) #batch

    def val_dataloader(self):
        return DataLoader(self.val, batch_size=64, num_workers=2)

    def test_dataloader(self):
        return DataLoader(self.test, batch_size=64, num_workers=2)

###Inizializzazione

In [ ]:
def run_experiment(seme):
    pl.seed_everything(seme, workers=True) #For Python `random`, NumPy, PyTorch, Torch CUDA, DataLoader workers
    MNIST_dm = MNISTDataModule(seme)
    modello = MLP()
    wandb_logger = WandbLogger(
    project=f"ML Prj",
    name=f"MLP_{seme}",
    notes = "MLP classico, file 'Prima_rete'",
    config={
        "seed": seme
    },
    log_model=True
)

    checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints/",
    save_top_k=1,
    monitor="valid_acc",
    mode="max"
)

    early_stopping = EarlyStopping(
    monitor="valid_acc",
    min_delta=0.0001,
    patience=6,
    verbose=False,
    mode="max"
)

    trainer = pl.Trainer(
    max_epochs = 15,
    callbacks=[checkpoint_callback, early_stopping],
    log_every_n_steps=20,
    logger=wandb_logger,
    deterministic=True
)
    trainer.fit(model=modello, datamodule=MNIST_dm)
    trainer.test(model=modello, datamodule=MNIST_dm)
    wandb.finish()

In [1]:
semi = (0, 42)

In [ ]:
for seme in semi:
    run_experiment(seme)

INFO:lightning_fabric.utilities.seed:Seed set to 0
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
100%|██████████| 9.91M/9.91M [00:00<00:00, 11.1MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 340kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 3.22MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 6.79MB/s]


INFO:pytorch_lightning.callbacks.model_summary:
  | Name      | Type               | Params | Mode 
---------------------------------------------------------
0 | train_acc | MulticlassAccuracy | 0      | train
1 | valid_acc | MulticlassAccuracy | 0      | train
2 | test_acc  | MulticlassAccuracy | 0      | train
3 | model     | Sequential         | 114 K  | train
---------------------------------------------------------
114 K     Trainable params
0         Non-trainable params
114 K     Total params
0.456     Total estimated model params size (MB)
18        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=15` reached.


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.9772999882698059     │
│         test_loss         │    0.0781741663813591     │
└───────────────────────────┴───────────────────────────┘

epoch,▁▁▁▁▁▁▁▂▂▂▂▃▃▃▃▃▃▃▃▃▄▄▅▅▅▅▆▆▆▆▇▇▇▇▇▇████
test_acc,▁
test_loss,▁
train_loss,▇█▆▆▄▅▅▂▄▇█▂▇█▄▃▃▄▅▁▂▃▄▃▄▃▁▃▄▁▄▂▂▃▂▁▂▂▃▂
trainer/global_step,▁▁▁▂▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▆▆▇▇██
valid_acc,▁▄▅▆▆▆▇▆█▇█▇███
valid_loss,█▆▄▃▃▃▂▃▂▂▂▁▁▁▁
epoch,15
test_acc,0.9773
test_loss,0.07817
train_loss,0.17147
